In [ ]:
!pwd

In [ ]:

root="/teamspace/studios/this_studio"
%cd $root
!git clone https://github.com/HeartMuLa/heartlib.git
%cd ./heartlib
!pip install -e .
!pip uninstall -y numpy scipy scikit-learn
!pip install numpy==1.26.4 scipy==1.11.4 scikit-learn==1.3.2
!pip install gradio
!hf download --local-dir './ckpt' 'HeartMuLa/HeartMuLaGen'
!hf download --local-dir './ckpt/HeartMuLa-oss-3B' 'HeartMuLa/HeartMuLa-oss-3B'
!hf download --local-dir './ckpt/HeartCodec-oss' 'HeartMuLa/HeartCodec-oss'

from IPython.display import clear_output
clear_output()

In [7]:
root="/teamspace/studios/this_studio"
%cd $root/heartlib
!python ./examples/run_music_generation.py --model_path=./ckpt --version="3B"

/teamspace/studios/this_studio/heartlib
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:01<00:00,  3.29it/s]
Device set to use cuda:0
100%|███████████████████████████████████████████| 10/10 [00:02<00:00,  3.82it/s]
Generated music saved to ./assets/output.mp3


In [15]:
root="/teamspace/studios/this_studio"
%cd $root/heartlib
import os
import uuid
import subprocess
import gradio as gr

# =========================
# PATHS
# =========================
ROOT = "/teamspace/studios/this_studio/heartlib"
SCRIPT = "./examples/run_music_generation.py"
MODEL_PATH = "./ckpt"
ASSETS_DIR = "./assets"

os.makedirs(ASSETS_DIR, exist_ok=True)

# =========================
# GENERATION FUNCTION
# =========================
def generate_music(
    lyrics_text,
    tags_text,
    max_length_sec,
    topk,
    temperature,
    cfg_scale,
):
    if not lyrics_text.strip():
        return None, "❌ Lyrics cannot be empty"

    uid = str(uuid.uuid4())

    lyrics_path = f"{ASSETS_DIR}/{uid}_lyrics.txt"
    tags_path = f"{ASSETS_DIR}/{uid}_tags.txt"
    output_path = f"{ASSETS_DIR}/{uid}_output.mp3"

    with open(lyrics_path, "w", encoding="utf-8") as f:
        f.write(lyrics_text)

    with open(tags_path, "w", encoding="utf-8") as f:
        f.write(tags_text)

    max_audio_length_ms = int(max_length_sec * 1000)

    cmd = [
        "python",
        SCRIPT,
        f"--model_path={MODEL_PATH}",
        "--version=3B",
        f"--lyrics={lyrics_path}",
        f"--tags={tags_path}",
        f"--save_path={output_path}",
        f"--max_audio_length_ms={max_audio_length_ms}",
        f"--topk={int(topk)}",
        f"--temperature={float(temperature)}",
        f"--cfg_scale={float(cfg_scale)}",
    ]

    try:
        subprocess.run(cmd, cwd=ROOT, check=True)
    except subprocess.CalledProcessError as e:
        return None, f"❌ Generation failed: {e}"

    return output_path, f"✅ Saved as {uid}_output.mp3"

# =========================
# GRADIO UI (LEFT → RIGHT)
# =========================
custom_css = """.gradio-container { font-family: 'SF Pro Display', -apple-system, BlinkMacSystemFont, sans-serif; }"""
with gr.Blocks(title="HeartMuLa Music Generator",theme=gr.themes.Soft(),
                              css=custom_css) as app:
    gr.Markdown("## 🎵 HeartMuLa Music Generator (CLI-Backed)")

    with gr.Row():
        # ========= LEFT SIDE =========
        with gr.Column(scale=2):
            lyrics_input = gr.Textbox(
                label="Lyrics",
                lines=10,
                placeholder="Type your lyrics here...",
            )

            tags_input = gr.Textbox(
                label="Tags / Style",
                lines=4,
                placeholder="genre: electronic, mood: dark, tempo: slow",
            )

            with gr.Accordion("⚙️ Generation Settings", open=False):
                max_length = gr.Slider(
                    minimum=10,
                    maximum=240,
                    value=240,
                    step=5,
                    label="Max Audio Length (seconds)",
                )

                topk = gr.Slider(
                    minimum=10,
                    maximum=200,
                    value=50,
                    step=1,
                    label="Top-K",
                )

                temperature = gr.Slider(
                    minimum=0.1,
                    maximum=2.0,
                    value=1.0,
                    step=0.05,
                    label="Temperature",
                )

                cfg_scale = gr.Slider(
                    minimum=0.5,
                    maximum=5.0,
                    value=1.5,
                    step=0.1,
                    label="CFG Scale",
                )

            generate_btn = gr.Button("Generate Music 🎶", variant="primary")

        # ========= RIGHT SIDE =========
        with gr.Column(scale=1):
            audio_out = gr.Audio(
                type="filepath",
                label="Generated Audio",
            )

            status_out = gr.Textbox(
                label="Status",
                interactive=False,
            )

    generate_btn.click(
        fn=generate_music,
        inputs=[
            lyrics_input,
            tags_input,
            max_length,
            topk,
            temperature,
            cfg_scale,
        ],
        outputs=[audio_out, status_out],
    )

# =========================
# RUN
# =========================
if __name__ == "__main__":
    app.launch(share=True)


/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
/tmp/ipykernel_1743/388309006.py:71: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css. Please pass these parameters to launch() instead.
  with gr.Blocks(title="HeartMuLa Music Generator",theme=gr.themes.Soft(),


/teamspace/studios/this_studio/heartlib
* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://f87efac65aec520b9d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.37it/s]
Device set to use cuda:0
100%|██████████| 10/10 [00:02<00:00,  3.82it/s]


Generated music saved to ./assets/6faca5fe-45f8-4cb4-9a4d-c0d0aebdbfe0_output.mp3


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.82it/s]
Device set to use cuda:0
100%|██████████| 10/10 [00:02<00:00,  3.81it/s]


...

Generated music saved to ./assets/51ab6db6-c1d9-44f9-82d6-e20e3d7fe61e_output.mp3


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.82it/s]
Device set to use cuda:0
100%|██████████| 10/10 [00:02<00:00,  3.84it/s]


Generated music saved to ./assets/0cae4a14-4c36-493a-ae5c-fea175a51a1a_output.mp3


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.47it/s]
Device set to use cuda:0
100%|██████████| 10/10 [00:02<00:00,  3.83it/s]


Generated music saved to ./assets/e526d56d-e662-4cc4-add3-00961a879530_output.mp3
